# Imports

In [ ]:
# ============================================================================
# 03_modeling.ipynb
# Modeling and Evaluation for Diabetes Binary Classification
# Extended with Hyperparameter Tuning, Multiple Models, ANN, and Cost-Benefit Analysis
# Integrated with preprocessing module
# ============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.utils.class_weight import compute_class_weight
import xgboost as xgb
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.neural_network import MLPClassifier
import sys
import os
import warnings
warnings.filterwarnings('ignore')